In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pymysql
import time

### 1. Selenium으로 웹페이지 열기 ###
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 창 숨김
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
url = "https://direct.samsungfire.com/mall/PP030108_001.html#"
driver.get(url)
time.sleep(10)  # 페이지 로딩 대기

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

### 2. 특약사항 크롤링 ###
special_terms = [("ESG 할인특약", "https://direct.samsungfire.com/mall/PP030102_001.html?tab=pageDicount*0"),
    ("마일리지", "https://direct.samsungfire.com/mall/PP030102_001.html"),
    ("마일리지환급금 바로사용 서비스", "https://direct.samsungfire.com/mall/PP030102_027.html"),
    ("애니핏 착한걷기", "https://direct.samsungfire.com/mall/PP030102_028.html"),
    ("Eco 모빌리티 이용", "https://direct.samsungfire.com/mall/PP030102_013.html"),
    ("착한운전 할인특약", "https://direct.samsungfire.com/mall/PP030102_007.html?tab=pageDicount*0"),
    ("착한드라이브", "https://direct.samsungfire.com/mall/PP030102_007.html"),
    ("TMAP 착한운전", "https://direct.samsungfire.com/mall/PP030102_025.html"),
    ("커넥티드 블랙박스 착한운전", "https://direct.samsungfire.com/mall/PP030102_005.html"),
    ("커넥티드카 착한운전", "https://direct.samsungfire.com/mall/PP030102_009.html"),
    ("블랙박스 할인특약", "https://direct.samsungfire.com/mall/PP030102_002.html"),
    ("차량부속장치 할인특약", "https://direct.samsungfire.com/mall/PP030102_006.html?tab=pageDicount*0"),
    ("커넥티드카", "https://direct.samsungfire.com/mall/PP030102_006.html"),
    ("첨단안전장치 장착 차량", "https://direct.samsungfire.com/mall/PP030102_008.html"),
    ("후측방충돌방지장치 장착 차량", "https://direct.samsungfire.com/mall/PP030102_011.html"),
    ("헤드업디스플레이 장착 차량", "https://direct.samsungfire.com/mall/PP030102_014.html"),
    ("어라운드뷰 모니터 장착 차량", "https://direct.samsungfire.com/mall/PP030102_010.html"),
    ("윈터타이어 장착", "https://direct.samsungfire.com/mall/PP030102_012.html"),
    ("운전자 할인특약", "https://direct.samsungfire.com/mall/PP030102_003.html?tab=pageDicount*0"),
    ("자녀사랑", "https://direct.samsungfire.com/mall/PP030102_003.html"),
    ("나눔 친서민", "https://direct.samsungfire.com/mall/PP030102_026.html"),
    ("무사고 할인", "https://direct.samsungfire.com/mall/PP030102_004.html")]
ul = soup.find('ul', class_='ui-menu-d3')

if ul:
    for li in ul.find_all('li'):
        a_tag = li.find('a')
        if a_tag and a_tag.text.strip():
            name = a_tag.text.strip()
            link = "https://direct.samsungfire.com" + a_tag['href']
            special_terms.append((name, link))
            print("크롤링된 특약 수:", len(special_terms))
            print("예시:", special_terms[:3])  # 앞 3개만 미리보기
            print("DB에 저장할 특약 수:", len(special_terms))

else:
    print("❌ 특약사항을 찾지 못했습니다.")

driver.quit()

### 3. MySQL 연결 및 테이블 생성 ###
conn = pymysql.connect(
    host='localhost',       # 또는 DB 주소
    user='playdata',            # 사용자명
    password='1111',     # 본인 비밀번호
    db='보험DB',            # 데이터베이스 이름
    charset='utf8mb4'
)
cursor = conn.cursor()

# 테이블 생성 (없으면 생성)
cursor.execute("""
CREATE TABLE IF NOT EXISTS 할인특약 (
    id INT AUTO_INCREMENT PRIMARY KEY,
    이름 VARCHAR(255),
    링크 TEXT
)
""")

### 4. 데이터 삽입 ###
for name, link in special_terms:
    cursor.execute("INSERT INTO 할인특약 (이름, 링크) VALUES (%s, %s)", (name, link))

conn.commit()
cursor.close()
conn.close()

print("✅ 특약사항이 DB에 저장되었습니다!")


크롤링된 특약 수: 23
예시: [('ESG 할인특약', 'https://direct.samsungfire.com/mall/PP030102_001.html?tab=pageDicount*0'), ('마일리지', 'https://direct.samsungfire.com/mall/PP030102_001.html'), ('마일리지환급금 바로사용 서비스', 'https://direct.samsungfire.com/mall/PP030102_027.html')]
DB에 저장할 특약 수: 23
✅ 특약사항이 DB에 저장되었습니다!


In [14]:
import pymysql
import pandas as pd

# 1. DB 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',  # ← 본인의 DB 비밀번호 입력
    db='보험DB',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor  # 딕셔너리 형태로 받기
)

# 2. 쿼리 실행 → 데이터프레임으로 받기
query = "SELECT * FROM 할인특약"
df = pd.read_sql(query, conn)

# 3. CSV로 저장
df.to_csv("할인특약_리스트.csv", index=False, encoding='utf-8-sig')

# 4. 완료 메시지
print("✅ CSV 파일로 저장 완료: 할인특약_리스트.csv")

# 5. 연결 종료
conn.close()



✅ CSV 파일로 저장 완료: 할인특약_리스트.csv


C:\Users\Playdata\AppData\Local\Temp\ipykernel_5156\1581527994.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import time

# 1. Selenium 설정 (브라우저 자동 조작)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 2. FAQ 페이지 접속
url = "https://www.kbinsure.co.kr/CG602010001.ec#none"
driver.get(url)
time.sleep(3)  # JavaScript 로딩 대기

# 3. 페이지 내용 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 4. FAQ 항목 찾기
faq_list = soup.select("li")  # 모든 li를 가져와서 필터링

faq_data = []

for li in faq_list:
    try:
        number = li.select_one("dl.first dt").text.strip()
        question = li.select_one("div.note1").text.strip()
        answer_html = li.select_one("div.closeTxtBox")
        answer = answer_html.get_text(separator="\n", strip=True)
        faq_data.append([number, question, answer])
    except Exception as e:
        continue  # 오류 시 해당 항목 건너뜀

driver.quit()

# 5. CSV로 저장
with open("faq_kbinsure.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["번호", "질문", "답변"])
    writer.writerows(faq_data)

print("✅ 크롤링 및 CSV 저장 완료! (faq_kbinsure.csv)")
       

✅ 크롤링 및 CSV 저장 완료! (faq_kbinsure.csv)
